In [1]:
import os
ROOT = os.path.join('..', '..')
import sys
sys.path.append(ROOT)

import numpy as np

from scripts.data import ml_ratings_df, ml_movies_df, ml_users_df, ml_train_test_split
from src.models.item_item import ItemItemCollaborativeFiltering

ml_ratings_df = ml_ratings_df.collect().to_pandas()
ml_users_df = ml_users_df.collect().to_pandas()
ml_movies_df = ml_movies_df.collect().to_pandas()

C:\Users\38097\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Train - Test Split

In [2]:
MAX_K = 30

ml_ratings_train_df, ml_ratings_test_df = ml_train_test_split(ml_ratings_df, MAX_K, 970_000_000)

# Item-Item Colaborative Filtering

In [3]:
recommender = ItemItemCollaborativeFiltering(ml_movies_df, ml_users_df)
recommender.fit(ml_ratings_train_df)

C:\Users\38097\DataspellProjects\UCU_5K2S_Recommender_Systems_Project\src\models\item_item.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.ratings_matrix.fillna(0, inplace=True)


In [4]:
metrics = recommender.evaluate(ml_ratings_test_df, MAX_K, verbose=False)

100%|██████████| 2500/2500 [06:58<00:00,  5.98it/s]


In [5]:
print("Average Precision@K:", round(np.mean(metrics['precision@k']), 4))
print("Average Recall@K:", round(np.mean(metrics['recall@k']), 4))
print("Average F1@K:", round(np.mean(metrics['f1@k']), 4))
print()
print("Best Precision@K:", round(np.max(metrics['precision@k']), 4))
print("Best Recall@K:", round(np.max(metrics['recall@k']), 4))
print("Best F1@K:", round(np.max(metrics['f1@k']), 4))

Average Precision@K: 0.4365
Average Recall@K: 0.1034
Average F1@K: 0.1496

Best Precision@K: 1.0
Best Recall@K: 0.439
Best F1@K: 0.507
